### Cohort Analysis Demo
This notebook demonstrates how to:
1. Generate a SQL query defining a cohort using PhenoML experimental cohort API
2. Query the cohort data from BigQuery
3. Analyze patient demographics, conditions, and medications

In [ ]:
from cohort_api import CohortAPI
from google.cloud import bigquery
import os

In [ ]:
client_id = os.getenv('CLIENT_ID')
client_secret = os.getenv('CLIENT_SECRET')

In [ ]:
cohort_api = CohortAPI(client_id, client_secret)

In [ ]:
query = "over 40 and prescribed statins"

In [ ]:
# Get the cohort SQL query from PhenoML API. Set exclude_deceased to True or False as desired for your analysis
cohort_response_sql = cohort_api.get_cohort_response(search_query=query, exclude_deceased=True, verbose=False)['sql']

In [ ]:
# For a more verbose exploration of the specific codes that are generated
cohort_response = cohort_api.get_cohort_response(search_query=query, exclude_deceased=True, verbose=True)
cohort_response_sql = cohort_response['sql']

In [ ]:
# Look at queries if desired
for query in cohort_response['queries']:
    print(f"\nResource: {query['resource']}")
    print(f"Exclude: {query['exclude']}")
    print(f"Search Params: {query['searchParams']}")

# Look at extracted codes
if 'code_extracts' in cohort_response:
    print("\nExtracted Codes:")
    for code in cohort_response['code_extracts']:
        print(f"{code['resource']} - {code['description']} ({code['code']})")

In [ ]:
# Use this dictionary to map the SQL output to your specific dataset and project ID. Synthea public BQ data provided for demo purposes
bq_fhir_table_mapping = {
    "PROJECT_ID": "bigquery-public-data",
    "DATASET": "fhir_synthea",
    "PATIENT_TABLE": "patient",
    "CONDITION_TABLE": "condition",
    "MEDICATIONREQUEST_TABLE": "medication_request"
}

bq_fhir_query = cohort_response_sql.format(**{key: f"{value}" for key, value in bq_fhir_table_mapping.items()})

In [ ]:
# Initialize BQ client. 
# This code assumes you're running this notebook with the necessary permissions to the BigQuery datasets you're querying 
# (e.g. via Vertex AI Workbench notebook instance with instance service account with required roles/permissions)
client = bigquery.Client()

query_job = client.query(bq_fhir_query)

# Fetch the results and extract IDs
results = query_job.result()
cohort_ptid_list = [row.id for row in results]

print(len(cohort_ptid_list), ": total patients found that meet cohort description")


In [ ]:
# Define the query parameters that will be used for the subsequent queries
job_config = bigquery.QueryJobConfig(
    query_parameters=[
        bigquery.ArrayQueryParameter("patient_ids", "STRING", cohort_ptid_list)
    ]
)

In [ ]:
##Analyze cohort: Patient Demographic Analysis

#Query template for Patient table. Replace the fields with the ones you need for your analysis
pt_query_template = """
SELECT
  id, 
  gender,
  birthdate
FROM
  {PROJECT_ID}.{DATASET}.{PATIENT_TABLE} AS Patient
WHERE
  id IN UNNEST(@patient_ids)
"""

# Replace placeholders in the query with your project details
pt_query = pt_query_template.format(
    PROJECT_ID=bq_fhir_table_mapping["PROJECT_ID"],
    DATASET=bq_fhir_table_mapping["DATASET"],
    PATIENT_TABLE=bq_fhir_table_mapping["PATIENT_TABLE"]
)


# Execute the query
pt_df = client.query(pt_query, job_config=job_config).to_dataframe()

pt_df.head()

In [ ]:
##Analyze cohort: Conditions 

#Query template for Condition table. Replace the fields with the ones you need for your analysis
cond_query_template = """
SELECT
  subject.patientid, 
  coding.code, 
  coding.display,
  coding.system, 
  onset.dateTime
FROM
  {PROJECT_ID}.{DATASET}.{CONDITION_TABLE} AS Condition, 
  UNNEST(Condition.code.coding) AS coding
WHERE
  subject.patientid IN UNNEST(@patient_ids)
"""

# Replace placeholders in the query
cond_query = cond_query_template.format(
    PROJECT_ID=bq_fhir_table_mapping["PROJECT_ID"],
    DATASET=bq_fhir_table_mapping["DATASET"],
    CONDITION_TABLE=bq_fhir_table_mapping["CONDITION_TABLE"]
)

# Execute the query
cond_df = client.query(cond_query, job_config=job_config).to_dataframe()
cond_df.head()


In [ ]:
##Analyze cohort: Medications

#Query template for Medication Request table. Replace the fields with the ones you need for your analysis
meds_query_template = """
SELECT
  subject.patientid, 
  coding.code, 
  coding.display,
  coding.system, 
  authoredOn
FROM
  {PROJECT_ID}.{DATASET}.{MEDICATIONREQUEST_TABLE} AS MedicationRequest, 
  UNNEST(MedicationRequest.medication.codeableConcept.coding) AS coding
WHERE
  subject.patientid IN UNNEST(@patient_ids)
"""

# Replace placeholders in the query
meds_query = meds_query_template.format(
    PROJECT_ID=bq_fhir_table_mapping["PROJECT_ID"],
    DATASET=bq_fhir_table_mapping["DATASET"],
    MEDICATIONREQUEST_TABLE=bq_fhir_table_mapping["MEDICATIONREQUEST_TABLE"]
)

# Execute the medications query
meds_df = client.query(meds_query, job_config=job_config).to_dataframe()
meds_df.head()
